In [1]:
import numpy as np
import pandas as pd

In [34]:
# 1️ Load the dataset

df = pd.read_csv("E:\\رواد الرقمية\\project\\railway.csv")

In [35]:
df.head()

,Transaction ID,Date of Purchase,Time of Purchase,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,Arrival Destination,Date of Journey,Departure Time,Arrival Time,Actual Arrival Time,Journey Status,Reason for Delay,Refund Request
0,da8a6ba8-b3dc-4677-b176,2023-12-08,12:41:11,Online,Contactless,Adult,Standard,Advance,43,London Paddington,Liverpool Lime Street,2024-01-01,11:00:00,13:30:00,13:30:00,On Time,NaN,No
1,b0cdd1b0-f214-4197-be53,2023-12-16,11:23:01,Station,Credit Card,Adult,Standard,Advance,23,London Kings Cross,York,2024-01-01,09:45:00,11:35:00,11:40:00,Delayed,Signal Failure,No
2,f3ba7a96-f713-40d9-9629,2023-12-19,19:51:27,Online,Credit Card,NaN,Standard,Advance,3,Liverpool Lime Street,Manchester Piccadilly,2024-01-02,18:15:00,18:45:00,18:45:00,On Time,NaN,No
3,b2471f11-4fe7-4c87-8ab4,2023-12-20,23:00:36,Station,Credit Card,NaN,Standard,Advance,13,London Paddington,Reading,2024-01-01,21:30:00,22:30:00,22:30:00,On Time,NaN,No
4,2be00b45-0762-485e-a7a3,2023-12-27,18:22:56,Online,Contactless,NaN,Standard,Advance,76,Liverpool Lime Street,London Euston,2024-01-01,16:45:00,19:00:00,19:00:00,On Time,NaN,No


In [ ]:
# 2 EDA

print("Shape:", df.shape)
df.info()

Shape: (31653, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31653 entries, 0 to 31652
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Transaction ID       31653 non-null  object
 1   Date of Purchase     31653 non-null  object
 2   Time of Purchase     31653 non-null  object
 3   Purchase Type        31653 non-null  object
 4   Payment Method       31653 non-null  object
 5   Railcard             10735 non-null  object
 6   Ticket Class         31653 non-null  object
 7   Ticket Type          31653 non-null  object
 8   Price                31653 non-null  int64 
 9   Departure Station    31653 non-null  object
 10  Arrival Destination  31653 non-null  object
 11  Date of Journey      31653 non-null  object
 12  Departure Time       31653 non-null  object
 13  Arrival Time         31653 non-null  object
 14  Actual Arrival Time  29773 non-null  object
 15  Journey Status       31653 non-nul

In [7]:
df.describe()

,Price
count,31653.000000
mean,23.439200
std,29.997628
min,1.000000
25%,5.000000
50%,11.000000
75%,35.000000
max,267.000000


In [36]:
print("Missing values (Nulls) for each column:")
print(df.isnull().sum())

Missing values (Nulls) for each column:
Transaction ID             0
Date of Purchase           0
Time of Purchase           0
Purchase Type              0
Payment Method             0
Railcard               20918
Ticket Class               0
Ticket Type                0
Price                      0
Departure Station          0
Arrival Destination        0
Date of Journey            0
Departure Time             0
Arrival Time               0
Actual Arrival Time     1880
Journey Status             0
Reason for Delay       27481
Refund Request             0
dtype: int64


In [ ]:
#need special library
#import sweetviz as sv
#report = sv.analyze(df)  # your DataFrame
#report.show_html("eda_report.html")

#file:0Code/eda_report.html
///C:/Users/Mohamed%20Saleh/AppData/Local/Programs/Microsoft%20VS%2

c:\Users\Mohamed Saleh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


Report eda_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [37]:
#3- Convert date columns

df["Date of Purchase"] = pd.to_datetime(df["Date of Purchase"], errors="coerce")
df["Date of Journey"] = pd.to_datetime(df["Date of Journey"], errors="coerce")


In [38]:
#4- Convert time columns

time_cols = ["Time of Purchase", "Departure Time", "Arrival Time", "Actual Arrival Time"]

for col in time_cols:
    df[col] = pd.to_datetime(df[col], format="%H:%M:%S", errors="coerce").dt.time


In [49]:
# 4️ Standarize the format(Clean categorical columns)

cat_cols = [
    "Purchase Type", "Payment Method", "Railcard", "Ticket Class",
    "Ticket Type", "Journey Status", "Refund Request","Departure Station", "Arrival Destination"
]

for col in cat_cols:
    df[col] = df[col].astype(str).str.strip().str.title()


# Map similar reasons to unified names
df["Reason for Delay"] = df["Reason for Delay"].str.strip().str.lower()

df["Reason for Delay"] = df["Reason for Delay"].replace({
    "weather conditions": "weather",
    "signal failure": "signal failure",
    "signalfailure": "signal failure",
    "staff shortage": "staffing",
    "staffshortage": "staffing",
    "technical issue": "technical issue",
    "traffic": "traffic",
    "none": "none",
})
df["Reason for Delay"] = df["Reason for Delay"].astype(str).str.strip().str.title()

In [40]:
# 5- Fill missing values 

# Reason for Delay column fill with None

df["Reason for Delay"] = df["Reason for Delay"].fillna("None")

# for Railcard column fill with No Railcard
df['Railcard'] = df['Railcard'].replace('Nan', 'No Railcard').fillna('No Railcard')

# we can not fill nulls in Actual Arrival Time because of the data type that my lead to wrong calculation 
#if we filled it with mean medin mode .... 
#and we can not drop it because it represent canceled journey

In [41]:
df.head()

,Transaction ID,Date of Purchase,Time of Purchase,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,Arrival Destination,Date of Journey,Departure Time,Arrival Time,Actual Arrival Time,Journey Status,Reason for Delay,Refund Request
0,da8a6ba8-b3dc-4677-b176,2023-12-08,12:41:11,Online,Contactless,Adult,Standard,Advance,43,London Paddington,Liverpool Lime Street,2024-01-01,11:00:00,13:30:00,13:30:00,On Time,None,No
1,b0cdd1b0-f214-4197-be53,2023-12-16,11:23:01,Station,Credit Card,Adult,Standard,Advance,23,London Kings Cross,York,2024-01-01,09:45:00,11:35:00,11:40:00,Delayed,Signal Failure,No
2,f3ba7a96-f713-40d9-9629,2023-12-19,19:51:27,Online,Credit Card,No Railcard,Standard,Advance,3,Liverpool Lime Street,Manchester Piccadilly,2024-01-02,18:15:00,18:45:00,18:45:00,On Time,None,No
3,b2471f11-4fe7-4c87-8ab4,2023-12-20,23:00:36,Station,Credit Card,No Railcard,Standard,Advance,13,London Paddington,Reading,2024-01-01,21:30:00,22:30:00,22:30:00,On Time,None,No
4,2be00b45-0762-485e-a7a3,2023-12-27,18:22:56,Online,Contactless,No Railcard,Standard,Advance,76,Liverpool Lime Street,London Euston,2024-01-01,16:45:00,19:00:00,19:00:00,On Time,None,No


In [42]:
#  Safely combine Date + Time 
def combine_date_time(row, time_col):
    """Combine Date of Journey and a time column into a full datetime"""
    if pd.notna(row['Date of Journey']) and pd.notna(row[time_col]):
        return pd.Timestamp.combine(row['Date of Journey'].date(), row[time_col])
    else:
        return pd.NaT

df['Departure Time'] = df.apply(lambda x: combine_date_time(x, 'Departure Time'), axis=1)
df['Arrival Time'] = df.apply(lambda x: combine_date_time(x, 'Arrival Time'), axis=1)
df['Actual Arrival Time'] = df.apply(lambda x: combine_date_time(x, 'Actual Arrival Time'), axis=1)



# If arrival is before departure -->> add 1 day (crossed midnight)
df.loc[df['Arrival Time'] < df['Departure Time'], 'Arrival Time'] += pd.Timedelta(days=1)
df.loc[df['Actual Arrival Time'] < df['Departure Time'], 'Actual Arrival Time'] += pd.Timedelta(days=1)

In [44]:
# Journey duration (in minutes)
df["Planned Journey Duration"] = ((df["Arrival Time"] - df["Departure Time"]).dt.total_seconds() / 60)


# Delay duration (in minutes)
df["Actual Journey Duration"] = df.apply(
    lambda x: ((x["Actual Arrival Time"] - x["Departure Time"]).total_seconds() / 60)
    if pd.notna(x["Actual Arrival Time"]) and pd.notna(x["Departure Time"])
    else None,
    axis=1
)


df["Delay (Minutes)"] = df.apply(
    lambda x: ((x["Actual Arrival Time"] - x["Arrival Time"]).total_seconds() / 60)
    if pd.notna(x["Actual Arrival Time"]) and pd.notna(x["Arrival Time"])
    else None,
    axis=1
)



In [45]:
#4- Convert time columns again but to become only time

time_cols = ["Time of Purchase", "Departure Time", "Arrival Time", "Actual Arrival Time"]

for col in time_cols:
    df[col] = pd.to_datetime(df[col], format="%H:%M:%S", errors="coerce").dt.time


In [46]:
df['Price Category'] = pd.cut(df['Price'], bins=[0, 20, 50, 100,300], labels=['Low', 'Medium', 'High','Very High'])

In [54]:
# Count number of passengers per journey
passenger_counts = (
    df.groupby(['Date of Journey', 'Departure Station', 'Arrival Destination', 'Departure Time'])
    .size()
    .reset_index(name='Passenger Count')
)

# Merge back to main DataFrame if you want this info in every row
df = df.merge(passenger_counts, on=['Date of Journey', 'Departure Station', 'Arrival Destination', 'Departure Time'], how='left')

# Display result


In [55]:
df[['Date of Journey', 'Departure Station', 'Arrival Destination', 'Departure Time', 'Passenger Count']].head(20)

,Date of Journey,Departure Station,Arrival Destination,Departure Time,Passenger Count
0,2024-01-01,London Paddington,Liverpool Lime Street,11:00:00,1
1,2024-01-01,London Kings Cross,York,09:45:00,1
2,2024-01-02,Liverpool Lime Street,Manchester Piccadilly,18:15:00,1
3,2024-01-01,London Paddington,Reading,21:30:00,1
4,2024-01-01,Liverpool Lime Street,London Euston,16:45:00,1
5,2024-01-01,London Kings Cross,York,06:15:00,2
6,2024-01-01,London Euston,Oxford,22:30:00,1
7,2024-01-01,Liverpool Lime Street,Manchester Piccadilly,00:00:00,1
8,2024-01-01,London Euston,York,00:00:00,1
9,2024-01-01,London Paddington,Reading,01:30:00,1


In [47]:
df.head(20)

,Transaction ID,Date of Purchase,Time of Purchase,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,...,Departure Time,Arrival Time,Actual Arrival Time,Journey Status,Reason for Delay,Refund Request,Planned Journey Duration,Actual Journey Duration,Delay (Minutes),Price Category
0,da8a6ba8-b3dc-4677-b176,2023-12-08,12:41:11,Online,Contactless,Adult,Standard,Advance,43,London Paddington,...,11:00:00,13:30:00,13:30:00,On Time,None,No,150.0,150.0,0.0,Medium
1,b0cdd1b0-f214-4197-be53,2023-12-16,11:23:01,Station,Credit Card,Adult,Standard,Advance,23,London Kings Cross,...,09:45:00,11:35:00,11:40:00,Delayed,Signal Failure,No,110.0,115.0,5.0,Medium
2,f3ba7a96-f713-40d9-9629,2023-12-19,19:51:27,Online,Credit Card,No Railcard,Standard,Advance,3,Liverpool Lime Street,...,18:15:00,18:45:00,18:45:00,On Time,None,No,30.0,30.0,0.0,Low
3,b2471f11-4fe7-4c87-8ab4,2023-12-20,23:00:36,Station,Credit Card,No Railcard,Standard,Advance,13,London Paddington,...,21:30:00,22:30:00,22:30:00,On Time,None,No,60.0,60.0,0.0,Low
4,2be00b45-0762-485e-a7a3,2023-12-27,18:22:56,Online,Contactless,No Railcard,Standard,Advance,76,Liverpool Lime Street,...,16:45:00,19:00:00,19:00:00,On Time,None,No,135.0,135.0,0.0,High
5,4e1dcd88-3d95-44ef-99fa,2023-12-30,07:56:06,Online,Credit Card,No Railcard,Standard,Advance,35,London Kings Cross,...,06:15:00,08:05:00,08:05:00,On Time,None,No,110.0,110.0,0.0,Medium
6,1c74479d-85a4-4ba1-a607,2023-12-31,00:02:01,Station,Credit Card,Adult,Standard,Advance,2,London Euston,...,22:30:00,23:40:00,23:40:00,On Time,None,No,70.0,70.0,0.0,Low
7,febf8dab-f808-46fa-bf2b,2023-12-31,01:35:18,Station,Contactless,Disabled,Standard,Advance,2,Liverpool Lime Street,...,00:00:00,00:30:00,00:30:00,On Time,None,No,30.0,30.0,0.0,Low
8,01df916f-4291-41ec-a37d,2023-12-31,01:43:09,Station,Credit Card,No Railcard,Standard,Advance,37,London Euston,...,00:00:00,01:50:00,02:07:00,Delayed,Signal Failure,No,110.0,127.0,17.0,Medium
9,a8cedba7-1923-459d-b046,2023-12-31,03:05:52,Online,Credit Card,No Railcard,Standard,Advance,13,London Paddington,...,01:30:00,02:30:00,02:30:00,On Time,None,No,60.0,60.0,0.0,Low


In [50]:
path_for_save="E:\\رواد الرقمية\\project\\railway_cleaned_finallllll2.csv"

df.to_csv(path_for_save, index=False)
